# **Accelerometer Based Alcohol Consumption Analysis**

In this project, I explored analyzing accelerometer data from smartphones to identify patterns linked to instances of heavy alcohol consumption. By applying advanced computational methods to calculate permutation entropy and complexity, I sought to uncover subtle variations in motion data that could indicate changes in alcohol intake levels.

**Objective**

The main objective was to create a powerful analytical tool that utilizes cutting-edge algorithms to process accelerometer data. This tool is intended to distinguish between sober and heavy drinking states, providing insights into an individual's drinking patterns and enabling early intervention when appropriate.

**Methodology**

**1. Data Acquisition and Preprocessing**

I created a comprehensive dataset by combining all transdermal alcohol content (TAC) readings into a single dataframe and merging the accelerometer data with the TAC readings after converting the time format from milliseconds to seconds. This preparatory step resulted in a rich dataset that captures both motion dynamics and quantifiable alcohol consumption levels.

**2. Exploratory Data Analysis (EDA)**

Utilizing visualization techniques, I explored the accelerometer data across the three axes in conjunction with the TAC values for each participant. This exploration was crucial in understanding the structure of the data and laying the groundwork for the subsequent analysis.


**3. Feature Extraction and Analysis**

At the core of my analytical endeavor was the computation of permutation entropy and complexity. I employed a sophisticated module to calculate these metrics, which measure the disorder and structural complexity within time series data. This approach provided a nuanced understanding of motion patterns in relation to alcohol consumption.


**4. Results Aggregation and Visualization**

By aggregating the calculated metrics and correlating them with TAC levels, I could discern patterns indicative of heavy drinking. The results were visualized through scatter plots, effectively illustrating the relationship between motion complexity and alcohol intake.

**Step 1**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
from IPython.display import display,clear_output
from scipy.stats import entropy
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [19]:
# loading all the datasets from the drive to dataframes
acc=pd.read_csv('/all_accelerometer_data_pids_13.csv')
phone=pd.read_csv('/content/phone_types.csv')

In [20]:
acc

,time,pid,x,y,z
0,0,JB3156,0.000000,0.000000,0.000000
1,0,CC6740,0.000000,0.000000,0.000000
2,1493733882409,SA0297,0.075800,0.027300,-0.010200
3,1493733882455,SA0297,-0.035900,0.079400,0.003700
4,1493733882500,SA0297,-0.242700,-0.086100,-0.016300
...,...,...,...,...,...
1467491,1493744721043,JB3156,-0.400972,-0.413745,-2.239268
1467492,1493744721043,MJ8002,0.001300,-0.002200,-0.003300
1467493,1493744721044,SF3079,0.012900,0.016200,0.022000
1467494,1493744721055,BK7610,-0.000300,0.003400,0.002000


**Step 2**

The 13 TAC reading files are concatenated to one dataframe.

In [21]:
#loading all the TAC files into a single dataframe along with their PIDs
concatenated_df = pd.concat(
    (pd.read_csv(f'/var/clean_tac/{y}_clean_TAC.csv').assign(pid=y) for y in phone['pid']),
    axis=0
)

In [22]:
concatenated_df.to_csv('/var/clean_tac.csv')
# Storing the concatenated data as a csv for verification
concatenated_df['pid'].nunique()
#verification

13

In [23]:
#converting accelerometer time readings from milliseconds to seconds.
acc['time']=acc['time']//1000

**Step 3**

The transdermal alcohol content (TAC) datasets and the accelerometer dataset were combined into a single dataframe.

In [24]:
# performing an inner join of the acceleormeter readings and the concatenated TAC readings
df_merged=pd.merge(acc,concatenated_df,left_on=['pid','time'],right_on=['pid','timestamp'])

**Step 4**


*   Plotting the sampled x, y, z accelerometer values and the corresponding transdermal alcohol content (TAC) values for all the users. The graphs for the selected participant ID (pid) are displayed.
*   Storing the sampled values of x, y, z accelerometer data and TAC values into a dictionary, with 13 keys - one for each participant ID (pid).

In [26]:
# Get unique PIDs
unique_pids = df_merged['pid'].unique()
# Function to plot data based on selected PID
def plot_data(pid):
    # Clear previous output
    clear_output(wait=True)

    # Get the selected PID data
    pid_data = df_merged[df_merged['pid'] == pid]
    time = np.arange(pid_data['y'].size) / 40  #sampling the data to rectify the frequency mismatch in TAC and accelerometer readings

    # Create new figure and subplots
    fig, ax = plt.subplots(4, figsize=(15, 8))

    # Plot accelerometer data
    ax[0].plot(time, pid_data['x'], label='X-axis')
    ax[1].plot(time, pid_data['y'], label='Y-axis')
    ax[2].plot(time, pid_data['z'], label='Z-axis')

    # Plot TAC readings
    ax[3].plot(time, pid_data['TAC_Reading'], label='TAC Reading', color='g')

    # Add horizontal line at TAC = 0.08 to indicate the legal intoxication limit
    ax[3].axhline(y=0.08, color='r', linestyle='--')

    # Add legend to each subplot
    for i in range(4):
        ax[i].legend()

    # Show the plot
    plt.xlabel("Time in seconds")
    plt.tight_layout()
    plt.show()



# Create interactive dropdown widget for selecting PID
interact(plot_data, pid=Dropdown(options=unique_pids, description='PID:'))



interactive(children=(Dropdown(description='PID:', options=('SA0297', 'PC6771', 'BK7610', 'JR8022', 'JB3156', …

<function __main__.plot_data(pid)>

In [27]:
filtered_dict={}
for i in df_merged['pid'].unique():
  filtered=df_merged[df_merged['pid']==i]

  time=np.arange(filtered['x'].size)/40
  filtered_data = {
        'time': time,
        'x': filtered['x'].values,
        'y': filtered['y'].values,
        'z': filtered['z'].values,
        'TAC': filtered['TAC_Reading'].values
    }

    # Add the DataFrame to the dictionary
  filtered_dict[i] = pd.DataFrame(filtered_data)

**Step 5**



*   Calculating complexity and entropy metrics for each user's sampled data stored in the filtered dictionary. The final dataframe includes columns for the participant ID (pid), x-axis entropy, x-axis complexity, y-axis entropy, y-axis complexity, z-axis entropy, z-axis complexity, and transdermal alcohol content (TAC).

*   Plotting a line graph between {axis}_entropy and {axis}_complexity with TAC values for the selected pid.

*   Plotting a scatterplot between x_entropy and y_complexity with TAC values. The size of the points determine the TAC value and the color indicates whether its above(Green) or below 0.08(Blue).



In [28]:
def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector representing the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector representing the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of ordinal patterns'''
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts)

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*(m-1)):, :]
    permutation = np.argsort(partition)
    idx = _hash(permutation)

    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = (idx == i).sum()
    return list(counts[counts != 0].astype(int))

def _hash(x):
    m, n = x.shape
    if n == 1:
        return np.zeros(m)
    return np.sum(np.apply_along_axis(lambda y: y < x[:, 0], 0, x), axis=1) * np.math.factorial(n-1) + _hash(x[:, 1:])


def p_entropy(op):
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)

def complexity(op):
    ''' This function computes the complexity of a time series defined as: Comp_JS = Q_o * JSdivergence * pe
    Q_o = Normalizing constant
    JSdivergence = Jensen-Shannon divergence
    pe = permutation entopry
    ARGS: ordinal pattern'''
    pe = p_entropy(op)
    constant1 = (0.5+((1 - 0.5)/len(op)))* np.log(0.5+((1 - 0.5)/len(op)))
    constant2 = ((1 - 0.5)/len(op))*np.log((1 - 0.5)/len(op))*(len(op) - 1)
    constant3 = 0.5*np.log(len(op))
    Q_o = -1/(constant1+constant2+constant3)

    temp_op_prob = np.divide(op, sum(op))
    temp_op_prob2 = (0.5*temp_op_prob)+(0.5*(1/len(op)))
    JSdivergence = (s_entropy(temp_op_prob2) - 0.5 * s_entropy(temp_op_prob) - 0.5 * np.log(len(op)))
    Comp_JS = Q_o * JSdivergence * pe
    return(Comp_JS)

def weighted_ordinal_patterns(ts, embdim, embdelay):
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts)

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*(m-1)):, :]
    xm = np.mean(partition, axis=1)
    weight = np.mean((partition - xm[:, None])**2, axis=1)
    permutation = np.argsort(partition)
    idx = _hash(permutation)
    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = sum(weight[i == idx])

    return list(counts[counts != 0])

results = []

for pid, df in filtered_dict.items():
    unique_tac_values = df['TAC'].unique()

    for tac_value in unique_tac_values:
        segment = df[df['TAC'] == tac_value]

        for axis in ['x', 'y', 'z']:
            # Use ordinal_patterns to get the patterns
            op = ordinal_patterns(segment[axis].values, embdim=3, embdelay=1)

            # Calculate permutation entropy using the new logic
            pe = p_entropy(op)

            # Calculate complexity using the new logic
            comp = complexity(op)

            # Append the results
            results.append({
                'pid': pid,
                f'{axis}_entropy': pe,
                f'{axis}_complexity': comp,
                'TAC': tac_value
            })

result_df = pd.DataFrame(results)
result_df = result_df[['pid', 'x_entropy', 'x_complexity', 'y_entropy', 'y_complexity', 'z_entropy', 'z_complexity', 'TAC']]



<ipython-input-28-2a49e77ea2cf>:37: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  return np.sum(np.apply_along_axis(lambda y: y < x[:, 0], 0, x), axis=1) * np.math.factorial(n-1) + _hash(x[:, 1:])
<ipython-input-28-2a49e77ea2cf>:28: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  counts = np.zeros(np.math.factorial(m))


In [29]:
result_df
# this has 486 rows since each row only has one axis value for each pid and TAC value

,pid,x_entropy,x_complexity,y_entropy,y_complexity,z_entropy,z_complexity,TAC
0,SA0297,0.937429,0.062262,NaN,NaN,NaN,NaN,0.032672
1,SA0297,NaN,NaN,0.939361,0.055267,NaN,NaN,0.032672
2,SA0297,NaN,NaN,NaN,NaN,0.929069,0.061885,0.032672
3,PC6771,0.950350,0.046908,NaN,NaN,NaN,NaN,0.069712
4,PC6771,NaN,NaN,0.962819,0.036135,NaN,NaN,0.069712
5,PC6771,NaN,NaN,NaN,NaN,0.970859,0.029228,0.069712
6,PC6771,0.953456,0.044712,NaN,NaN,NaN,NaN,0.074726
7,PC6771,NaN,NaN,0.982092,0.017516,NaN,NaN,0.074726
8,PC6771,NaN,NaN,NaN,NaN,0.970859,0.029228,0.074726
9,PC6771,0.993914,0.005933,NaN,NaN,NaN,NaN,0.077807


In [30]:
result_df.fillna(0,inplace=True)
# fill the NaN will zeros.

In [31]:
# Group the DataFrame by 'pid' and 'TAC', summing entropy and complexity for each axis.
result_df = result_df.groupby(['pid', 'TAC']).agg({
    'x_entropy': 'sum',
    'x_complexity': 'sum',
    'y_entropy': 'sum',
    'y_complexity': 'sum',
    'z_entropy': 'sum',
    'z_complexity': 'sum'
}).reset_index() # Reset index to keep DataFrame format

In [32]:
result_df
#486/3= 162 rows, only one row with all the axis complexity and entropy and the corresponding TAc values

,pid,TAC,x_entropy,x_complexity,y_entropy,y_complexity,z_entropy,z_complexity
0,BK7610,0.065357,0.900914,0.086503,0.875105,0.108894,0.961033,0.037932
1,BK7610,0.076462,0.989350,0.010338,0.986392,0.013538,0.983933,0.015965
2,BK7610,0.085197,0.991455,0.008398,0.991455,0.008398,0.991455,0.008398
3,BK7610,0.090165,0.805045,0.149129,0.761480,0.179115,0.827230,0.141411
4,CC6740,0.041870,0.912207,0.077751,0.745557,0.166311,0.960058,0.038633
5,CC6740,0.055006,0.962584,0.035234,0.987509,0.011908,0.975424,0.024996
6,HV0618,0.040862,0.846825,0.123038,0.925894,0.066849,0.894570,0.091516
7,JB3156,0.035549,0.913480,0.076273,0.829680,0.142484,0.897024,0.094082
8,JB3156,0.061969,0.919545,0.071478,0.888107,0.095696,0.855477,0.135259
9,JR8022,0.088046,0.982092,0.017516,0.963146,0.039533,0.966530,0.031911


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
from IPython.display import clear_output

# Get unique PIDs (You will need to replace result_df with your actual dataframe)
unique_pids = result_df['pid'].unique()

def plot_entropy_complexity_for_pid(pid):
    # Clear previous output
    clear_output(wait=True)

    # Filter data for the selected PID (You will need to replace result_df with your actual dataframe)
    pid_data = result_df[result_df['pid'] == pid]

    # Plotting each metric (x, y, z) vs TAC separately
    fig, axes = plt.subplots(3, 2, figsize=(15, 20))

    # Plot for X Entropy vs TAC
    axes[0, 0].plot(pid_data['TAC'], pid_data['x_entropy'], marker='o', linestyle='-', color='magenta')
    axes[0, 0].set_title('X Entropy vs TAC')
    axes[0, 0].set_xlabel('TAC')
    axes[0, 0].set_ylabel('X Entropy')
    axes[0, 0].grid(True)

    # Plot for Y Entropy vs TAC
    axes[1, 0].plot(pid_data['TAC'], pid_data['y_entropy'], marker='o', linestyle='-', color='cyan')
    axes[1, 0].set_title('Y Entropy vs TAC')
    axes[1, 0].set_xlabel('TAC')
    axes[1, 0].set_ylabel('Y Entropy')
    axes[1, 0].grid(True)

    # Plot for Z Entropy vs TAC
    axes[2, 0].plot(pid_data['TAC'], pid_data['z_entropy'], marker='o', linestyle='-', color='orange')
    axes[2, 0].set_title('Z Entropy vs TAC')
    axes[2, 0].set_xlabel('TAC')
    axes[2, 0].set_ylabel('Z Entropy')
    axes[2, 0].grid(True)

    # Plot for X Complexity vs TAC
    axes[0, 1].plot(pid_data['TAC'], pid_data['x_complexity'], marker='o', linestyle='-', color='magenta')
    axes[0, 1].set_title('X Complexity vs TAC')
    axes[0, 1].set_xlabel('TAC')
    axes[0, 1].set_ylabel('X Complexity')
    axes[0, 1].grid(True)

    # Plot for Y Complexity vs TAC
    axes[1, 1].plot(pid_data['TAC'], pid_data['y_complexity'], marker='o', linestyle='-', color='cyan')
    axes[1, 1].set_title('Y Complexity vs TAC')
    axes[1, 1].set_xlabel('TAC')
    axes[1, 1].set_ylabel('Y Complexity')
    axes[1, 1].grid(True)

     # Plot for Z Complexity vs TAC
    axes[2, 1].plot(pid_data['TAC'], pid_data['z_complexity'], marker='o', linestyle='-', color='orange')
    axes[2, 1].set_title('Z Complexity vs TAC')
    axes[2, 1].set_xlabel('TAC')
    axes[2, 1].set_ylabel('Z Complexity')
    axes[2, 1].grid(True)

    plt.tight_layout()
    plt.show()

# Create interactive dropdown widget for selecting PID
interact(plot_entropy_complexity_for_pid, pid=Dropdown(options=unique_pids, description='Select PID:'))

interactive(children=(Dropdown(description='Select PID:', options=('BK7610', 'CC6740', 'HV0618', 'JB3156', 'JR…

<function __main__.plot_entropy_complexity_for_pid(pid)>

In [38]:
import ipywidgets as widgets


# Assuming result_df is already loaded and processed as per your requirements

def plot_data(pid):
    # Filter the data for the selected PID
    result = result_df if pid == 'All' else result_df[result_df['pid'] == pid]

    # Use TAC values for point sizes (using an arbitrary scale for visualization)
    point_sizes = (result['TAC'] / result['TAC'].max()) * 1000

    # Determine point colors based on TAC values
    colors = np.where(result['TAC'] >= 0.08, 'green', 'blue')

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(result['x_entropy'], result['y_complexity'], s=point_sizes, c=colors, alpha=0.6)
    plt.title(f'Scatter Plot for PID: {pid}')
    plt.xlabel('X_Entropy')
    plt.ylabel('Y_Complexity')
    plt.grid(True)
    plt.show()

# Add an "All" option to view all PIDs
unique_pids_with_all = np.append(['All'], result_df['pid'].unique())

# Create a dropdown widget for selecting PIDs, including an "All" option
pid_dropdown = widgets.Dropdown(options=unique_pids_with_all, description='PID:', value='All')

# Display the widget and bind the interactive plot function
widgets.interactive(plot_data, pid=pid_dropdown)

interactive(children=(Dropdown(description='PID:', options=('All', 'BK7610', 'CC6740', 'HV0618', 'JB3156', 'JR…

# **Key Insights**

**Variability Across Axes**

- The permutation entropy and complexity metrics vary across the
x, y, and z accelerometer axes for the same transdermal alcohol content (TAC) levels. This suggests that the impact of alcohol on motion patterns is dependent on the specific direction of movement.


**Relationship Between TAC Levels and Motion Complexity**
- Observations indicate that as transdermal alcohol content (TAC) levels increase, movement complexity can either increase or decrease, suggesting alcohol has a differential impact on the predictability of movement.

**Insight into the Dynamics of Movement**
- Significant differences in entropy and complexity at similar TAC levels across the x, y, and z axes highlight alcohol's multidimensional effect on movement.

**Potential for Predictive Modeling**
- The computed metrics provide a dataset for developing models to predict TAC levels from movement complexity and entropy, which could be useful for real-time monitoring and interventions.

**Implications for Intervention Strategies**
- Analyzing movement patterns associated with heavy drinking can inform proactive intervention strategies to mitigate alcohol-related harm.

# **Conclusion**

In this project, I investigated the impact of alcohol consumption on human movement by analyzing accelerometer data using advanced entropy and complexity metrics. The analysis uncovered distinct patterns that correlate with alcohol levels, indicating the potential to develop models that can predict alcohol consumption based on movement data. This work presents new opportunities for leveraging technology to monitor and address problematic drinking behaviors, underscoring the value of sophisticated data analysis techniques in health and behavioral research.